# TAREFA DATASET GRUPO

Para esta fase, decidimos escolher um dataset referente a avaliações de vinhos. 
Pode ser encontrado neste link: https://www.kaggle.com/datasets/zynicide/wine-reviews <br>
O objetivo desta fase é, através deste dataset, inferir acerca do valor de points que será dado a um dado vinho.

### **1.** Importar as bibliotecas essenciais do Python para a elaboração desta tarefa

In [196]:
import sklearn as skl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('always')

### **2.** Carregar o dataset para um dataframe da biblioteca Panda 

In [ ]:
df = pd.read_csv('docs/winemag-data-130k-v2.csv', encoding="utf-8", skipinitialspace=True)

### **3.** Obtenção de informação acerca do dataset: tipos de dados das features, conteúdo do dataset e estatística

* **tipos de dados das features**

In [ ]:
df.info()

A feature objetivo, points, está preenchida em todas as linhas.<br>
Muitas das outras features têm de ser preenchidas mais tarde ou retiradas completamente por estarem incompletas (como taster_twiter_handle)

* **conteúdo do dataset**

In [ ]:
df

Reparamos que a feature Unnamed: 0 apenas serve como id, sendo portanto desnecessária. <br>
Region 1 e 2 sao apenas especificações de province, dependendo do seu número de valores únicos, talvez sejam um overload de informação que é melhor excluir. <br>
Da mesma forma, taster_twitter_handle apenas complementa taster_name.

* **estatística**

In [ ]:
df.describe()

Podemos ver que embora points seja da perspetiva de fora um atributo de classificação que iria de 0-100, neste dataset apenas temos valores entre 80 e 100.<br>
O valor máximo de preço é muito superior à sua média (possível necessidade de tratar de outliers).


* **Distribuição da feature target**

In [ ]:
df.points.hist()

Distribuição do target é aproximadamente normal, o que será benéfico para a criação de modelos, como por exemplo de regressão linear.

* **Distribuição da feature price**

In [ ]:
df.price.hist(bins=[0,20,50,100,200,300,1000,2000,2500,3000])

Através do grafo dos preços (possível segunda feature alvo), podemos ver que a grande maioria destes se distribui por volta de valores menores que 100.

* **Análise dos valores únicos das features**

In [ ]:
for c in df:
    print(f"{c}: {df[c].unique()}")
    print(f"Quantidade: {df[c].nunique()}")
    print("---------------------------------------")

| Feature               | Justificação                                                                                                                                          |
|-----------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|
| Unnamed               | Como mencionado anteriormente, esta feature serve como id, devendo ser retirada.                                                                      |
| description           | O número de valores únicos é muito elevado, além de serem strings sem nenhum significado para os modelos                                              |
| designation           | Número de valores únicos muito elevados (para strings).                                                                                               |
| province              | Embora uma especificação de country, como não tem um número excessivo de valores únicos, pode ser benéfico.                                           |
| region_1              | Especificação de especificação (province), com muitos valores únicos. Provavelmente melhor retirar.                                                   |
| region_2              | Nova especificação de province.                                                                                                                       |
| taster_twitter_handle | Especificação/AlterEgo do taster_name, informação redundante.                                                                                         |
| title                 | Como mencionado anteriormente, número de únicos demasiado elevado, mas contém informação útil dentro deste, devendo ser retirada nos próximos passos. |
| variety               | Número elevado de strings únicas, pode ou não ser relevante, devendo ser experimentado o modelo em ambos os casos.                                    |
| winery                | Número muito elevado de strings únicas, embora potencialmente relevantes numa perspetiva de mundo real.                                               |

### **4.** Preparação dos dados

In [ ]:
df.head()

In [ ]:
df.columns

* **Remoção de features**

In [ ]:
remove_features_list = ["Unnamed: 0",'designation', "description",'region_1', "region_2", "taster_twitter_handle","winery"]
for ft in remove_features_list:
    df = df.drop(ft, axis=1)

* **Preenchimento valores em falta**

In [ ]:
df.price.fillna(df.price.mean(),inplace =True)
df.country.fillna(str(df.country.mode()),inplace =True)
df.province.fillna(str(df.province.mode()),inplace =True)
df.taster_name.fillna('unknown',inplace =True)
df.variety.fillna(str(df.variety.mode()),inplace =True)

* **Tratamento de title**

Obsevando previamente os valores da feature title, observamos que todos parecem incluir o ano de produção de vinho e, embora a atualmente esta feature apresente demasiados valores únicos para ser útil, se conseguirmos retirar apenas o ano desta, possívelmente poderá ser usado no modelo.

In [ ]:
df[df.title.str.contains(r'.*\d{4}.*')].info()

Confirmamos que a grande maioria das linhas apresenta o ano. (apenas 4609 de 130k não apresentam)

In [ ]:
df.title = df.title.str.replace(r'(.|\n)*(\d{4})(.|\n)*',r'\2',regex=True)
df.title = df.title[df.title.str.contains(r'\d{4}')]


In [ ]:
df.info()

Mudar nome da feature para year

In [ ]:
rename_map = {'title':'year'}
df.rename(columns=rename_map,inplace=True)

In [ ]:
df

Preencher valores nulos com a moda

In [ ]:
df.year.fillna(df.year.mode().astype(int).values[0],inplace=True)

In [ ]:
df.year = df.year.astype(int)

* **Remoção de duplicados**

In [ ]:
# Remoção de registos duplicados (caso hajam)
df.drop_duplicates(inplace=True)

In [ ]:
df.info()

* **Labeling das features**

Número de valores únicos e tipo de cada feature

In [ ]:
columns = df.columns.values
for c in columns:
    print(f"{c} : {df[c].nunique()}  \n   type : {df[c].dtype}")

In [ ]:
df.info()

Labeling de features tipo object

In [ ]:
for c in df.columns.values:
    if(df[c].dtype=='object'):
        print(c)
        labels = df[c].astype('category').cat.categories.tolist()
        replace_map_comp = {c : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}
        df.replace(replace_map_comp,inplace=True)

df.head()


### **5.** Aplicação de modelos de Machine Learning

#### 5.1. Decision Tree regressor

Imports necessários

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

Retirar target do data-set de treino e criar uma cópia do data-set, para verificar predictions

In [ ]:
x = df.drop(['points'],axis=1)
y = df['points'].to_frame()

In [ ]:
x

In [ ]:
y

Separar o data-set em conjuntos de treino e teste <br>
Tamanho de teste - 25% <br>
Seed = 2022

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=2022)

Criar modelo de árvores de decisão

In [ ]:
clf = DecisionTreeRegressor(random_state=2022)

Treinar modelo

In [ ]:
clf.fit(x_train,y_train)

Gerar previsões

In [ ]:
predictions = clf.predict(x_test)
pd.DataFrame(predictions)

MAE

In [ ]:
mean_absolute_error(y_test,predictions)

In [ ]:
y_test

K-Cross validation com K = 10

In [ ]:
from sklearn.model_selection import cross_val_score

N_FOLDS = 10
scores = cross_val_score(clf, x, y, cv=N_FOLDS)
print(scores)
print("Accuracy: %0.2f" %(scores.mean()))
print("Desvio padrão: %0.2f" %(scores.std()))

#### 5.2. Linear Regression

Importar as funções necessárias para este modelo

In [151]:
from sklearn.linear_model import LinearRegression

Separar o data-set em conjuntos de treino e teste <br>
Tamanho de teste - 25% <br>
Seed = 2022

In [163]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2022)

Criar instância de LinearRegression()

In [164]:
lm = LinearRegression()

In [165]:
lm.fit(x_train, y_train)

LinearRegression()

Gerar previsões

In [183]:
predictions = lm.predict(x_test)
df_pred = pd.DataFrame(data=predictions)
df_pred

,0
0,87.554118
1,88.986521
2,90.100114
3,87.671286
4,87.633931
...,...
29361,87.725749
29362,87.671178
29363,89.079302
29364,88.175809


In [187]:
y_test

,points
105320,87
100128,90
1139,84
40948,86
110663,90
...,...
129921,91
109424,80
77237,93
77868,85


#### **Avaliação do modelo**

In [188]:
from sklearn import metrics
from math import sqrt

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

MAE: 2.2825673584524777
MSE: 8.462452909482172
RMSE: 2.909029547715556


#### 5.3. Support Vector Machine

Separar o data-set em conjuntos de treino e teste <br>
Tamanho de teste - 25% <br>
Seed = 2022

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=2022)

Importar funções necessárias para este modelo

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

Criação de uma instância SVC()

In [ ]:
model = SVC(random_state=2022)
model.fit(x_train,y_train)

Geração de previsões

In [ ]:
y_pred = model.predict(x_test)
y_pred

In [ ]:
y_test

**Avaliação do modelo**

Importar funções necessárias

In [ ]:
from sklearn.metrics import classification_report, plot_confusion_matrix

Matriz de confusão

In [ ]:
plot_confusion_matrix(model, x_test, y_test) 

Classification report

In [ ]:
print(classification_report(y_test, y_pred))

TODO: 
- usar GRIDSEARCH e PRUNNIG?
- Redes neuronais